In [1]:
import logging
logging.basicConfig(filename = '../logging/BrainNetsLog.log', 
                    filemode='w', 
                    format = '%(asctime)s %(name)s %(lineno)d %(funcName)s %(levelname)s:%(message)s', 
                    level = logging.INFO)
logger = logging.getLogger('Jupyter Notebook')

In [2]:
import BrainNets as bn
reload(bn)
import dataOperator as do
reload(do)

import time
import theano
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline 

/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6020 on context None
Mapped name None to device cuda0: GeForce GTX 750 Ti (0000:01:00.0)


In [3]:
net = bn.Network()
trainFunc = net.trainFunction()
valAndTestFunc = net.valAndTestFunction()

In [35]:
trainCubes = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', 
                           sourceDataDirectory = '../data/SimpleExploreSomeData/', 
                           cubeSize = 33,
                           trainOrVal = 'val',
                           onTheFly = True)
valCubes = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', 
                         sourceDataDirectory = '../data/SimpleExploreSomeData/',
                         cubeSize = 33,
                         onTheFly = True, 
                         trainOrVal = 'val')

In [36]:
numEpochs = 100

In [37]:
trainingLoss = []
validationLoss = []
validationACC = []

In [38]:
for epoch in range(numEpochs):
    logger.info('{} th of {} epochs'.format(epoch, numEpochs))
    
    trainError = 0
    startTime = time.time()
    trainBatch = 0
    
#     for batch in synthsisTrain.generateSynthesisData(1000, group0 = (0,5), group1 = (1, 5)):
    for batch in trainCubes.fetchCubes(batchSize = 100, parallel = True):
        inputs, targets = batch
        trainLoss = trainFunc(inputs, targets)

        trainError += trainLoss
        trainBatch += 1

    valError = 0
    valACC = 0
    valBatches = 0
        
#     for batch in synthsisVal.generateSynthesisData(1000, group0 = (1,5), group1 = (1, 5)):
    for batch in valCubes.fetchCubes(batchSize = 100, parallel = True):
        inputs, targets = batch
        targets = targets
        valAndTestLoss, valAndTestACC = valAndTestFunc(inputs, targets)
        valError += valAndTestLoss
        valACC += valAndTestACC
        valBatches += 1
        
    print "Epoch {} of {} took {:.3f}s".format(epoch + 1, numEpochs, time.time() - startTime)
    print "  training loss:\t\t{:.6f}".format(trainError / trainBatch)
    print "  validation loss:\t\t{:.6f}".format(valError / valBatches)
    print "  validation accuracy:\t\t{:.2f} %".format(valACC / valBatches * 100)
    
    trainingLoss.append(trainError / trainBatch)
    validationLoss.append(valError / valBatches)
    validationACC.append(valACC / valBatches * 100)

ValueError: GpuReshape: trying to reshape an array of total size 4877800 into an array of total size 200.
Apply node that caused the error: GpuReshape{2}(GpuCrossentropySoftmax1HotWithBiasDx.0, Join.0)
Toposort index: 149
Inputs types: [GpuArrayType<None>(float32, matrix), TensorType(int64, vector)]
Inputs shapes: [(100, 48778), (2,)]
Inputs strides: [(195112, 4), (8,)]
Inputs values: ['not shown', array([100,   2])]
Outputs clients: [[InplaceGpuDimShuffle{0,x,x,x,1}(GpuReshape{2}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
plt.plot(trainingLoss)
plt.ylabel('trainingLoss')
plt.show()

In [ ]:
plt.plot(validationLoss)
plt.ylabel('validationLoss')
plt.show()

In [ ]:
plt.plot(validationACC)
plt.ylabel('validationACC')
plt.show()

In [ ]:
theano.printing.debugprint(trainFunc)

In [ ]:
%time do.normalizeDataSet(dataPath = '../data/trainParallelNorm/', normalizedDataDir = '../data/outPutPara1/',parallel = False)

In [ ]:
%time do.normalizeDataSet(dataPath = '../data/trainParallelNorm/', normalizedDataDir = '../data/outPutPara2/',parallel = True)

In [32]:
do.makeCubesOnTheFly(dataPath = '../data/SimpleExploreSomeData/', 
             cubeDirectory = '../data/tryGenerateCubeData1/',
             cubeSize = 33, 
             parallel = False)

In [33]:
do.makeCubesOnTheFly(dataPath = '../data/SimpleExploreSomeData/', 
             cubeDirectory = '../data/tryGenerateCubeData/',
             cubeSize = 33, 
             parallel = True)

In [ ]:
synthsisTrain = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', batchSize = 1000)
synthsisVal = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', trainOrVal = 'val', batchSize = 1000)

In [ ]:
startTime = time.time()
for inputs, targets in trainCubes.fetchCubes(batchSize = 100, parallel = False):
    print time.time() - startTime
    startTime = time.time()

In [ ]:
startTime = time.time()
for inputs, targets in trainCubes.fetchCubes(batchSize = 100, parallel = True):
    print time.time() - startTime
    startTime = time.time()